# Grundlagen der Multimediatechnik
## Wintersemester 2021/2022

## Übungslatt 10
<b>Wichtig</b>: Bitte kennzeichnen Sie Ihre Abgabe mit Ihrem Namen und dem Namen Ihres Übungspartners. <br>
Laden Sie bitte Ihre Abgabe als PDF-/ZIP-Datei auf der ILIAS-Plattform hoch. Andere Dateiformate sowie Scans von handgeschriebenen Abgaben werden nicht gewertet. Achten Sie darauf, nur kompilierbaren, kommentierten Code abzugeben. Nicht-kompilierbarer Code wird mit <b>0 Punkten</b> bewertet! \\
Alle Abgaben müussen folgender Namenskonvention entsprechen:  `gmt_uebungXX_nachname1_nachname2.format` <br>
Im Falle Ihres Notebooks: `gmt_uebungXX_nachname1_nachmame2.ipynb`

In [1]:
# imports

import numpy as np
import cv2
from PIL import Image
import time

### Aufgabe 2 - Edge Change Ratio (13 Punkte)
Implementieren Sie nun eine einfache Schnitterkennung nach der in der Vorlesung vorgestellten Edge Change Ratio (ECR) Methode für das `tagesschau.gif`. <br>
Um die Schnitterkennung durchführen zu können, müssen die einzelnen Frames des GIFs in Graustufenbilder konvertiert werden. Dieser Prozess erfolgt in der Funktion `grey_image_list()`, die eine Liste der Graustufenbilder zurückgibt. Hierfür benötigen Sie die Pythonbibliothek `pillow`.

In [2]:
def grey_image_list(gif):
    image_list = []
    for frame in range(gif.n_frames):
        gif.seek(frame)
        image_list.append(np.asarray(gif.convert("L")))
    return image_list

1. Im nächsten Schritt implementieren Sie die Funktion `edge_detection()`, die für jeden Frame die Kantenerkennung durchführt und ebenfalls eine Liste von Frames zurückgibt. Berechnen Sie dabei zuerst die Gradienten, um die Gradientmagnitude herleiten zu können. Führen Sie anschließend eine Schwellwertfilterung durch. Die Schwellwertfilterung ist definiert als:
$$ T_{global}(g) = \begin{cases}
    0 & \text{wenn}\ g < 35\\
    255 & \text{wenn}\ g \geq 35
\end{cases}$$
Sie können für die Gradientenberechnung die `gradient()`-Funktion von `numpy` nutzen.

In [3]:
def edge_detection(image_list):

    frame_list =[]

    for image in image_list: 
        G_x = np.gradient(image, axis=0)
        
        G_y = np.gradient(image, axis=1)
        M = np.round(np.sqrt( (G_x* G_x) + (G_y* G_y) ))
        
        M[M<35] = 0
        M[M>=35] = 255  

        frame_list.append(M)
        
    return frame_list
    

2. Vervollständigen Sie nun die Funktion `ecr()`. Hierfür berechnen Sie die Anzahl der Pixel der Kanten, welche bei Frame-Wechsel hinzukommen $(E_{in})$ oder wegfallen $(E_{out})$, sowie die Anzahl der Kantenpixel im aktuellen $(s_i)$ und im vorhergehenden Bild $(s_{i-1})$.
$$\mathrm{ECR}_{i-1} = \max\left(\frac{E_{in}}{s_{i-1}}, \frac{E_{out}}{s_{i}}\right)$$
Liegt der Wert über 1, so ist ein Schnitt zu erkennen.

In [4]:
def ecr(image_list):

    def edge_number(image): 
        
        x = (image == 255)
        number = len(image[x])

        return number 
    
    cut = []
    
    for i in range(len(image_list)-1): 
        
        
        #Anzahl der Kantenpixel in vorherigem Bild
        image_j = image_list[i]
        s_j = edge_number(image_j)

        #Anzahl der Kantenpixel in aktuellem Bild 
        image_i = image_list[i+1]
        s_i = edge_number(image_i)

        #E_in 

        diff_in = image_i - image_j 
        e_in = edge_number(diff_in)

        #E_out

        diff_out = image_j - image_i 
        e_out = edge_number(diff_out)

        ecr_i = max(e_in/s_j , e_out/s_i)

        if(ecr_i >= 1): 
            cut.append(i)

    # TODO
    return cut









In [5]:
gif = Image.open('tagesschau_intro.gif')

#since = time.time()

greyscale_images = grey_image_list(gif)
edges = edge_detection(greyscale_images)
cut = ecr(edges)

#time_elapsed = time.time() - since
#print('Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Schnitt erkannt bei Frame: ', cut)


Schnitt erkannt bei Frame:  [158, 159]


Bei welchem Frame-Übergang erkennt Ihr Programm einen Schnitt?

Das Programm erkennt einen Schnitt zwischen Frame 158 und 159.